In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [2]:
Map = geemap.Map()

In [8]:
hydrosheds = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_2')

scenes1 = ee.FeatureCollection('users/ekyzivat/order-1-coords')
scenes2 = ee.FeatureCollection('users/ekyzivat/order-2-coords')
scenes3 = ee.FeatureCollection('users/ekyzivat/order-3-coords')
scenes4 = ee.FeatureCollection('users/ekyzivat/order-4-coords')
scenes = scenes1.merge(scenes2).merge(scenes3).merge(scenes4)

def buffer_100km(feature):
    return feature.buffer(100000)

buffered_scenes = scenes.map(buffer_100km)

In [9]:
spatialFilter_nonbuff = ee.Filter.intersects(leftField = '.geo', rightField = '.geo', maxError = 1000)
invertedJoin_nonbuff = ee.Join.inverted()
invertedJoined_nonbuff = invertedJoin_nonbuff.apply(hydrosheds, scenes, spatialFilter_nonbuff)

#spatialFilter_buff = ee.Filter.intersects(leftField = '.geo', rightField = '.geo', maxError = 1000)
#invertedJoin_buff = ee.Join.inverted()
#invertedJoined_buff = invertedJoin_buff.apply(hydrosheds, buffered_scenes, spatialFilter_buff)

#spatialFilter_questioning = ee.Filter.intersects(leftField = '.geo', rightField = '.geo', maxError = 1000)
#invertedJoin_questioning = ee.Join.inverted()
#invertedJoined_questioning  = invertedJoin_questioning.apply(invertedJoined_buff, invertedJoined_nonbuff, spatialFilter_questioning)

#saveAllJoin = ee.Join.saveAll(matchesKey = 'scenes')
#aveAllJoined = saveAllJoin.apply(hydrosheds, scenes, spatialFilter)

#hydrosheds_scenes = invertedJoined.merge(saveAllJoined)

#def scene_per_shed(shed):
#    nScenes = ee.List(shed.get('scenes')).size()
#    return shed.set('n_scenes', nScenes)

#hydrosheds_scenes = hydrosheds_scenes.map(scene_per_shed)

In [10]:
Map.addLayer(invertedJoined_nonbuff,{'color': 'black'}, 'non buffered')
#Map.addLayer(invertedJoined_buff,{'color': 'red'}, 'buffered')
#Map.addLayer(invertedJoined_questioning,{'color': 'green'}, 'question regions')

Map.addLayer(scenes, {'color': 'red'})

In [6]:
print('Number of missing sheds:', invertedJoined_nonbuff.size().getInfo())

Number of missing sheds: 8


In [7]:
Map.addLayerControl()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…